In [0]:
import networkx as nx
import matplotlib.pyplot as plt
import json
import random
import math
from priority_queue import *
from ucs import UCSearch

# 1. Let's define some useful functions and constants.

In [0]:
#@title
POLLUTION_COEFF = 0.00001 # The pollution cost coefficient

def loadGraph(filename):
  with open(filename) as labyrinth_file:
    dict_labyrinth = json.load(labyrinth_file)
    return dict_labyrinth, nx.Graph(dict_labyrinth)

def insertKey(graphDict, keyName):
  """Inserts a keyword inside the graph's attributes.
  E.g: If london has the neighbours:
  {'london':{'bristol':{'weight': 100}},
            {'cambridge':{'weight':54}}}, calling this function will insert a new
  key in its neighbour's attributes. The result would be:
  {'london':{'bristol':{'weight': 100, '[keyName]': None}},
            {'cambridge':{'weight':54, '[keyName]': None}}}"""
  for key, val in graphDict.items():
    for key2, val2 in val.items():
      graphDict[key][key2][keyName] = None
  return graphDict

def initialiseGraph(graphDict):
  """Initialises the various neighbour attributes for the purposes of Question 1C.
  This function randomly initialises the velocity within the range [40, 100]. Once
  the velocity is initialised the travel time from one city to another is calculated
  as well as its corresponding travel time. This function returns a nxGraph instance
  of the dictionary read by the .json file. Moreover, the function assumes that the keys
  velocity, time and pollution_cost already exist."""
  for key, val in graphDict.items():
    for neigh_key, neigh_dict in val.items():
      for att_key, att_val in neigh_dict.items():
          graphDict[key][neigh_key]['velocity'] = 316.227 # km/h
          v = graphDict[key][neigh_key]['velocity']
          graphDict[key][neigh_key]['time'] = graphDict[key][neigh_key]['weight'] / graphDict[key][neigh_key]['velocity']
          t = graphDict[key][neigh_key]['time']
          w = graphDict[key][neigh_key]['weight']
          graphDict[key][neigh_key]['overall_cost'] = t + (POLLUTION_COEFF * w * v)
          
  return nx.Graph(graphDict)

def show_weighted_graph(networkx_graph, node_size=2500, font_size=12, fig_size=(16, 7)):
  # Allocate the given fig_size in order to have space for each node
  plt.figure(num=None, figsize=fig_size, dpi=80)
  plt.axis('off')
  # Compute the position of each vertex in order to display it nicely
  nodes_position = nx.spring_layout(networkx_graph) 
  # You can change the different layouts depending on your graph
  # Extract the weights corresponding to each edge in the graph
  edges_weights  = nx.get_edge_attributes(networkx_graph,'weight')
  # Draw the nodes (you can change the color)
  nx.draw_networkx_nodes(networkx_graph, nodes_position, node_size=node_size,  
                         node_color = ["orange"]*networkx_graph.number_of_nodes())
  # Draw only the edges
  nx.draw_networkx_edges(networkx_graph, nodes_position, 
                         edgelist=list(networkx_graph.edges), width=2)
  # Add the weights
  nx.draw_networkx_edge_labels(networkx_graph, nodes_position, 
                               edge_labels = edges_weights)
  # Add the labels of the nodes
  nx.draw_networkx_labels(networkx_graph, nodes_position, font_size=font_size, 
                          font_family='sans-serif')
  plt.axis('off')
  plt.show()


#2. Load the graph.
The `loadGraph` function returns the graph in the forms of a dictionary and of an nxGraph. The dictionary will be used to insert new keys in our graph.

In [0]:
uk_cities_dict, uk_cities_graph = loadGraph("UK_cities.json")

#3. Run the UCS algorithm
In this section, the UCS algorithm will be run to find the path that yields the minimum distance. Note that in the trace, time is shown as zero, because the dictionary does not yet have the key 'time'.

In [0]:
searcher = UCSearch(uk_cities_graph, 'weight')
searcher.ucs('london', 'aberdeen', True)

#4. Using the UCS algorithm to find the minimum pollution cost.
For the case of the environmentally-friendly friends, the loaded dictionary will be used to insert the keys "velocity", "time" and "pollution_cost", that are needed to calculate the pollution cost for the journey from one city to another. Note that the velocity from city A to city B is chosen randomly and it is a number in the range [0, 100].

In [0]:
uk_cities_dict_new = insertKey(uk_cities_dict, "velocity")
uk_cities_dict_new = insertKey(uk_cities_dict, "time")
uk_cities_dict_new = insertKey(uk_cities_dict, "overall_cost")

In [0]:
nxGraphCities = initialiseGraph(uk_cities_dict_new)

In [0]:
ucs_pollution = UCSearch(nxGraphCities, 'overall_cost')
ucs_pollution.ucs('london', 'aberdeen')